In [1]:
import pandas as pd
import os


# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()
from arcgis.geoanalytics import manage_data

from IPython.display import display, Markdown

In [2]:
working_directory = os.getcwd()
data_folder       = os.path.join(working_directory, 'data'   )
results_folder    = os.path.join(working_directory, 'results')

masternetwork = r"E:\GitHub\Transit-Map\data\Master_2021-11-15.shp"

In [4]:
data = [['BY_2019' ,'v832_2019_ODStationDetail.csv'],
        ['TIP_2024','v832_2024_ODStationDetail.csv'],
        ['RTP_2030','v832_2030_ODStationDetail.csv'],
        ['RTP_2040','v832_2040_ODStationDetail.csv'],
        ['RTP_2050','v832_2050_ODStationDetail.csv']]

dfTotVol = pd.DataFrame()

for df in data:
    dfTemp = pd.read_csv(os.path.join(data_folder, df[1]))
    dfTemp['MODEL'] = df[0]
    display(dfTemp)

    dfLinkLookup = dfTemp[['MODE','NAMEID','NAME','LINKSEQ1','A']].copy()
    dfLinkLookup['LINKSEQ_OF_B'] = dfLinkLookup['LINKSEQ1'] + 1

    dfLinkLookupMax = dfLinkLookup.groupby(['MODE','NAMEID','NAME'],as_index=False).agg(MAXLINKSEQ1=('LINKSEQ1','max'))
    dfLinkLookupMax['MAXLINKSEQp1'] = dfLinkLookupMax['MAXLINKSEQ1'] + 1
    dfLinkLookupWithMaxPlus1 = pd.DataFrame.merge(dfLinkLookup, dfLinkLookupMax, on=('MODE','NAMEID','NAME'), how='left')
    dfLinkLookupWithMaxPlus1.loc[(dfLinkLookupWithMaxPlus1['LINKSEQ1']==0), 'LINKSEQ1'] = dfLinkLookupWithMaxPlus1['MAXLINKSEQp1']
    dfLinkLookupWithMaxPlus1.loc[(dfLinkLookupWithMaxPlus1['LINKSEQ_OF_B']==1), 'LINKSEQ_OF_B'] = 0

    dfLinkLookupWithMaxPlus1 = dfLinkLookupWithMaxPlus1.drop(columns=['MAXLINKSEQ1','MAXLINKSEQp1'])

    dfJoinToGetB = pd.DataFrame.merge(dfLinkLookupWithMaxPlus1, dfLinkLookupWithMaxPlus1, left_on=('MODE','NAMEID','NAME','LINKSEQ_OF_B'), right_on=('MODE','NAMEID','NAME','LINKSEQ1'))
    dfJoinToGetB = dfJoinToGetB[['MODE','NAMEID','NAME','LINKSEQ1_x','A_x','A_y']]
    dfJoinToGetB.columns = ('MODE','NAMEID','NAME','LINKSEQ1','A','B')

    dfTempPrepped = pd.DataFrame.merge(dfJoinToGetB,dfTemp[['MODE','NAMEID','NAME','LINKSEQ1','A','DIST','D_TOT_VOL','MODEL']], on=('MODE','NAMEID','NAME','LINKSEQ1','A'))

    #display(dfLinkLookup)
    #display(dfLinkLookupMax)
    #display(dfLinkLookupWithMaxPlus1)
    #display(dfJoinToGetB)
    display(dfTempPrepped)

    dfTotVol = dfTotVol.append(dfTempPrepped)

    #display(dfLinkLookup.sort_values(by='LINK', ascending=False).drop_duplicates(subset='NAME'))

display(dfTotVol)
display(dfTotVol['D_TOT_VOL'].sum())

,MODE,NAMEID,NAME,OPERATOR,ONEWAY,LINKSEQ1,LINKSEQ2,A,STOP_NAME,STOPS,...,O_TOT_WLK,O_TOT_BRD,O_TOT_XIT,O_TOT_VOL,D_TOT_DRV,D_TOT_WLK,D_TOT_BRD,D_TOT_XIT,D_TOT_VOL,MODEL
0,4,1,D460,0,1,1,0,20131,NaN,1,...,0.00,0.00,0.00,0.00,1.9,2.60,4.50,4.50,9.00,BY_2019
1,4,1,D460,0,1,2,130,3551,NaN,0,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,9.00,BY_2019
2,4,1,D460,0,1,3,129,3434,NaN,1,...,0.00,0.00,0.00,0.00,0.0,0.36,0.36,0.36,9.69,BY_2019
3,4,1,D460,0,1,4,128,3458,NaN,0,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,9.69,BY_2019
4,4,1,D460,0,1,5,127,3971,NaN,1,...,0.00,0.00,0.00,0.00,0.0,0.44,0.44,0.44,10.27,BY_2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8598,9,109,BRT_ProOrm,0,0,81,5,25378,NaN,0,...,0.00,0.00,0.00,156.47,0.0,0.00,0.00,0.00,572.66,BY_2019
8599,9,109,BRT_ProOrm,0,0,82,4,22700,NaN,0,...,0.00,0.00,0.00,156.47,0.0,0.00,0.00,0.00,572.66,BY_2019
8600,9,109,BRT_ProOrm,0,0,83,3,22695,NaN,0,...,0.00,0.00,0.00,156.47,0.0,0.00,0.00,0.00,572.66,BY_2019
8601,9,109,BRT_ProOrm,0,0,84,2,24779,NaN,0,...,0.00,0.00,0.00,156.47,0.0,0.00,0.00,0.00,572.66,BY_2019


,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,MODEL
0,4,1,D460,1,20131,3551,0.11,9.00,BY_2019
1,4,1,D460,2,3551,3434,0.38,9.00,BY_2019
2,4,1,D460,3,3434,3458,0.39,9.69,BY_2019
3,4,1,D460,4,3458,3971,0.38,9.69,BY_2019
4,4,1,D460,5,3971,9042,0.03,10.27,BY_2019
...,...,...,...,...,...,...,...,...,...
8489,9,109,BRT_ProOrm,80,22710,25378,0.25,572.66,BY_2019
8490,9,109,BRT_ProOrm,81,25378,22700,0.12,572.66,BY_2019
8491,9,109,BRT_ProOrm,82,22700,22695,0.20,572.66,BY_2019
8492,9,109,BRT_ProOrm,83,22695,24779,0.12,572.66,BY_2019


,MODE,NAMEID,NAME,OPERATOR,ONEWAY,LINKSEQ1,LINKSEQ2,A,STOP_NAME,STOPS,...,O_TOT_WLK,O_TOT_BRD,O_TOT_XIT,O_TOT_VOL,D_TOT_DRV,D_TOT_WLK,D_TOT_BRD,D_TOT_XIT,D_TOT_VOL,MODEL
0,4,1.0,D460,0,1,1,0,20131,NaN,1,...,0.00,0.00,0.00,0.00,1.68,3.15,4.83,4.83,9.66,TIP_2024
1,4,1.0,D460,0,1,2,130,3551,NaN,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9.66,TIP_2024
2,4,1.0,D460,0,1,3,129,3434,NaN,1,...,0.00,0.00,0.00,0.00,0.00,0.24,0.24,0.24,10.10,TIP_2024
3,4,1.0,D460,0,1,4,128,3458,NaN,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.10,TIP_2024
4,4,1.0,D460,0,1,5,127,3971,NaN,1,...,0.00,0.00,0.00,0.00,0.00,0.34,0.34,0.34,10.45,TIP_2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8770,9,111.0,DavisSLCCon,0,0,109,5,8978,NaN,0,...,0.00,0.00,0.00,15.56,0.00,0.00,0.00,0.00,44.03,TIP_2024
8771,9,111.0,DavisSLCCon,0,0,110,4,8977,NaN,0,...,0.00,0.00,0.00,15.56,0.00,0.00,0.00,0.00,44.03,TIP_2024
8772,9,111.0,DavisSLCCon,0,0,111,3,8976,NaN,0,...,0.00,0.00,0.00,15.56,0.00,0.00,0.00,0.00,44.03,TIP_2024
8773,9,111.0,DavisSLCCon,0,0,112,2,3551,NaN,0,...,0.00,0.00,0.00,15.56,0.00,0.00,0.00,0.00,44.03,TIP_2024


,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,MODEL
0,4,1.0,D460,1,20131,3551,0.11,9.66,TIP_2024
1,4,1.0,D460,2,3551,3434,0.38,9.66,TIP_2024
2,4,1.0,D460,3,3434,3458,0.39,10.10,TIP_2024
3,4,1.0,D460,4,3458,3971,0.38,10.10,TIP_2024
4,4,1.0,D460,5,3971,9042,0.03,10.45,TIP_2024
...,...,...,...,...,...,...,...,...,...
8659,9,111.0,DavisSLCCon,108,8979,8978,0.10,44.03,TIP_2024
8660,9,111.0,DavisSLCCon,109,8978,8977,0.04,44.03,TIP_2024
8661,9,111.0,DavisSLCCon,110,8977,8976,0.13,44.03,TIP_2024
8662,9,111.0,DavisSLCCon,111,8976,3551,0.15,44.03,TIP_2024


,MODE,NAMEID,NAME,OPERATOR,ONEWAY,LINKSEQ1,LINKSEQ2,A,STOP_NAME,STOPS,...,O_TOT_WLK,O_TOT_BRD,O_TOT_XIT,O_TOT_VOL,D_TOT_DRV,D_TOT_WLK,D_TOT_BRD,D_TOT_XIT,D_TOT_VOL,MODEL
0,4,1.0,D460,0,1,1,0,20131,NaN,1,...,0.00,0.00,0.00,0.00,0.26,0.31,0.57,0.57,1.14,RTP_2030
1,4,1.0,D460,0,1,2,130,3551,NaN,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.14,RTP_2030
2,4,1.0,D460,0,1,3,129,3434,NaN,1,...,0.00,0.00,0.00,0.00,0.00,0.17,0.17,0.17,1.44,RTP_2030
3,4,1.0,D460,0,1,4,128,3458,NaN,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.44,RTP_2030
4,4,1.0,D460,0,1,5,127,3971,NaN,1,...,0.00,0.00,0.00,0.00,0.00,0.29,0.29,0.29,1.57,RTP_2030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9579,9,112.0,DavisSLCCon,0,0,109,5,8978,NaN,0,...,0.00,0.00,0.00,18.83,0.00,0.00,0.00,0.00,47.97,RTP_2030
9580,9,112.0,DavisSLCCon,0,0,110,4,8977,NaN,0,...,0.00,0.00,0.00,18.83,0.00,0.00,0.00,0.00,47.97,RTP_2030
9581,9,112.0,DavisSLCCon,0,0,111,3,8976,NaN,0,...,0.00,0.00,0.00,18.83,0.00,0.00,0.00,0.00,47.97,RTP_2030
9582,9,112.0,DavisSLCCon,0,0,112,2,3551,NaN,0,...,0.00,0.00,0.00,18.83,0.00,0.00,0.00,0.00,47.97,RTP_2030


...

,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,MODEL
0,4,1.0,D460,1,20131,3551,0.11,1.25,RTP_2040
1,4,1.0,D460,2,3551,3434,0.38,1.25,RTP_2040
2,4,1.0,D460,3,3434,3458,0.39,1.58,RTP_2040
3,4,1.0,D460,4,3458,3971,0.38,1.58,RTP_2040
4,4,1.0,D460,5,3971,9042,0.03,1.67,RTP_2040
...,...,...,...,...,...,...,...,...,...
10163,9,116.0,BRT_StateSt,190,23145,25104,0.17,533.52,RTP_2040
10164,9,116.0,BRT_StateSt,191,25104,23148,0.10,533.52,RTP_2040
10165,9,116.0,BRT_StateSt,192,23148,26488,0.09,533.52,RTP_2040
10166,9,116.0,BRT_StateSt,193,26488,26487,0.15,533.52,RTP_2040


,MODE,NAMEID,NAME,OPERATOR,ONEWAY,LINKSEQ1,LINKSEQ2,A,STOP_NAME,STOPS,...,O_TOT_WLK,O_TOT_BRD,O_TOT_XIT,O_TOT_VOL,D_TOT_DRV,D_TOT_WLK,D_TOT_BRD,D_TOT_XIT,D_TOT_VOL,MODEL
0,4,1,D460,0,1,1,0,20131,NaN,1,...,0.00,0.00,0.00,0.00,0.25,0.40,0.64,0.64,1.28,RTP_2050
1,4,1,D460,0,1,2,130,3551,NaN,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.28,RTP_2050
2,4,1,D460,0,1,3,129,3434,NaN,1,...,0.00,0.00,0.00,0.00,0.00,0.20,0.20,0.20,1.63,RTP_2050
3,4,1,D460,0,1,4,128,3458,NaN,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.63,RTP_2050
4,4,1,D460,0,1,5,127,3971,NaN,1,...,0.00,0.00,0.00,0.00,0.00,0.33,0.33,0.33,1.67,RTP_2050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11688,9,131,BRT_StateSt,0,0,191,5,25104,NaN,0,...,0.00,0.00,0.00,266.66,0.00,0.00,0.00,0.00,602.00,RTP_2050
11689,9,131,BRT_StateSt,0,0,192,4,23148,NaN,0,...,0.00,0.00,0.00,266.66,0.00,0.00,0.00,0.00,602.00,RTP_2050
11690,9,131,BRT_StateSt,0,0,193,3,26488,NaN,0,...,0.00,0.00,0.00,266.66,0.00,0.00,0.00,0.00,602.00,RTP_2050
11691,9,131,BRT_StateSt,0,0,194,2,26487,NaN,0,...,0.00,0.00,0.00,266.66,0.00,0.00,0.00,0.00,602.00,RTP_2050


,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,MODEL
0,4,1,D460,1,20131,3551,0.11,1.28,RTP_2050
1,4,1,D460,2,3551,3434,0.38,1.28,RTP_2050
2,4,1,D460,3,3434,3458,0.39,1.63,RTP_2050
3,4,1,D460,4,3458,3971,0.38,1.63,RTP_2050
4,4,1,D460,5,3971,9042,0.03,1.67,RTP_2050
...,...,...,...,...,...,...,...,...,...
11557,9,131,BRT_StateSt,190,23145,25104,0.17,602.00,RTP_2050
11558,9,131,BRT_StateSt,191,25104,23148,0.10,602.00,RTP_2050
11559,9,131,BRT_StateSt,192,23148,26488,0.09,602.00,RTP_2050
11560,9,131,BRT_StateSt,193,26488,26487,0.15,602.00,RTP_2050


,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,MODEL
0,4,1.0,D460,1,20131,3551,0.11,9.00,BY_2019
1,4,1.0,D460,2,3551,3434,0.38,9.00,BY_2019
2,4,1.0,D460,3,3434,3458,0.39,9.69,BY_2019
3,4,1.0,D460,4,3458,3971,0.38,9.69,BY_2019
4,4,1.0,D460,5,3971,9042,0.03,10.27,BY_2019
...,...,...,...,...,...,...,...,...,...
11557,9,131.0,BRT_StateSt,190,23145,25104,0.17,602.00,RTP_2050
11558,9,131.0,BRT_StateSt,191,25104,23148,0.10,602.00,RTP_2050
11559,9,131.0,BRT_StateSt,192,23148,26488,0.09,602.00,RTP_2050
11560,9,131.0,BRT_StateSt,193,26488,26487,0.15,602.00,RTP_2050


19624960.73

In [18]:
#AB-BAs exist
display(dfTotVol[(dfTotVol['A']==3403) & (dfTotVol['B']==3440)])
display(dfTotVol[(dfTotVol['A']==3440) & (dfTotVol['B']==3403)])

#only ABs
display(dfTotVol[(dfTotVol['A']==3611)])
display(dfTotVol[(dfTotVol['B']==3611)])

,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,MODEL
5538,4,68.0,S455,355,3403,3440,0.06,134.02,BY_2019
6055,4,71.0,S470,283,3403,3440,0.06,78.21,BY_2019
5394,4,65.0,S455,355,3403,3440,0.06,151.88,TIP_2024
5915,4,68.0,S470,283,3403,3440,0.06,85.61,TIP_2024
4904,4,59.0,S455,355,3403,3440,0.06,154.45,RTP_2030
5424,4,62.0,S470,283,3403,3440,0.06,89.49,RTP_2030
4879,4,58.0,S455,355,3403,3440,0.06,93.78,RTP_2040
5399,4,61.0,S470,283,3403,3440,0.06,43.97,RTP_2040
5006,4,58.0,S455,355,3403,3440,0.06,92.77,RTP_2050
5526,4,61.0,S470,283,3403,3440,0.06,43.96,RTP_2050


,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,MODEL
5293,4,68.0,S455,110,3440,3403,0.06,78.27,BY_2019
5888,4,71.0,S470,116,3440,3403,0.06,175.69,BY_2019
5149,4,65.0,S455,110,3440,3403,0.06,84.98,TIP_2024
5748,4,68.0,S470,116,3440,3403,0.06,148.78,TIP_2024
4659,4,59.0,S455,110,3440,3403,0.06,75.91,RTP_2030
5257,4,62.0,S470,116,3440,3403,0.06,137.16,RTP_2030
4634,4,58.0,S455,110,3440,3403,0.06,67.22,RTP_2040
5232,4,61.0,S470,116,3440,3403,0.06,77.79,RTP_2040
4761,4,58.0,S455,110,3440,3403,0.06,70.01,RTP_2050
5359,4,61.0,S470,116,3440,3403,0.06,82.00,RTP_2050


,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,MODEL
7717,6,100.0,S456,131,3611,3503,0.17,30.08,BY_2019
7724,6,100.0,S456,131,3611,3503,0.17,45.29,TIP_2024
8185,6,98.0,S456,131,3611,3503,0.17,48.60,RTP_2030
8659,6,101.0,S456,131,3611,3503,0.17,59.66,RTP_2040
9839,6,115.0,S456,131,3611,3503,0.17,77.80,RTP_2050


,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,MODEL
7716,6,100.0,S456,130,10246,3611,0.25,30.08,BY_2019
7723,6,100.0,S456,130,10246,3611,0.25,45.29,TIP_2024
8184,6,98.0,S456,130,10246,3611,0.25,48.60,RTP_2030
8658,6,101.0,S456,130,10246,3611,0.25,59.66,RTP_2040
9838,6,115.0,S456,130,10246,3611,0.25,77.80,RTP_2050


In [19]:
#create lookup for opposite direction

dfABs = dfTotVol.groupby(['A','B'],as_index=False).agg(COUNT=('MODE','size'))
#display(dfABs)

dfBAs = dfABs.rename(columns={'A':'B','B':'A'})
dfBAs = dfBAs[['A','B','COUNT']]
#display(dfBAs)

dfJointoBAs = pd.DataFrame.merge(dfTotVol,dfBAs,on=('A','B'))
dfBAsExistforABPairs = dfJointoBAs.groupby(['A','B'],as_index=False).agg(COUNT=('COUNT','size'))

dfBAsExistforABPairs = dfBAsExistforABPairs[dfBAsExistforABPairs['A'] > dfBAsExistforABPairs['B']]

dfBAsExistforABPairs['BAExist'] = 1
display(dfBAsExistforABPairs)

#shouldn't have BA
display(dfBAsExistforABPairs[dfBAsExistforABPairs['A']==3611])
#should have BA
display(dfBAsExistforABPairs[dfBAsExistforABPairs['A']==3403])




#if BA pair exists, then switch A and B so only AB combination in Table... Have to do it this way, instead of min, in case of one-way links where A may be >B
dfTotVol_NewAB = pd.DataFrame.merge(dfTotVol,dfBAsExistforABPairs,on=('A','B'),how='left')
dfTotVol_NewAB = dfTotVol_NewAB.fillna(0)

dfTotVol_NewAB['NewA'] = 0
dfTotVol_NewAB['NewB'] = 0

#switch AB or non-BA
dfTotVol_NewAB.loc[(dfTotVol_NewAB['BAExist']==1), 'NewA'] = dfTotVol_NewAB[['A','B']].min(axis=1)
dfTotVol_NewAB.loc[(dfTotVol_NewAB['BAExist']==1), 'NewB'] = dfTotVol_NewAB[['A','B']].max(axis=1)

#maitain AB or non-BA
dfTotVol_NewAB.loc[(dfTotVol_NewAB['BAExist']==0), 'NewA'] = dfTotVol_NewAB['A']
dfTotVol_NewAB.loc[(dfTotVol_NewAB['BAExist']==0), 'NewB'] = dfTotVol_NewAB['B']

display(dfTotVol_NewAB)

#should have switched ABs
display(dfTotVol_NewAB[(dfTotVol_NewAB['A']==3403) & (dfTotVol_NewAB['B']==3440)])
display(dfTotVol_NewAB[(dfTotVol_NewAB['A']==3440) & (dfTotVol_NewAB['B']==3403)])

#should only have AB
display(dfTotVol_NewAB[(dfTotVol_NewAB['A']==3611)])

,A,B,COUNT,BAExist
9,3414,3411,14,1
19,3421,3419,5,1
43,3439,3430,17,1
45,3440,3403,10,1
48,3442,3421,5,1
...,...,...,...,...
4958,31334,20262,2,1
4959,31334,31333,2,1
4960,31412,9551,20,1
4962,31413,6885,21,1


,A,B,COUNT,BAExist


,A,B,COUNT,BAExist


,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,MODEL,COUNT,BAExist,NewA,NewB
0,4,1.0,D460,1,20131,3551,0.11,9.00,BY_2019,29.0,1.0,3551,20131
1,4,1.0,D460,2,3551,3434,0.38,9.00,BY_2019,10.0,1.0,3434,3551
2,4,1.0,D460,3,3434,3458,0.39,9.69,BY_2019,0.0,0.0,3434,3458
3,4,1.0,D460,4,3458,3971,0.38,9.69,BY_2019,0.0,0.0,3458,3971
4,4,1.0,D460,5,3971,9042,0.03,10.27,BY_2019,0.0,0.0,3971,9042
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48355,9,131.0,BRT_StateSt,190,23145,25104,0.17,602.00,RTP_2050,0.0,0.0,23145,25104
48356,9,131.0,BRT_StateSt,191,25104,23148,0.10,602.00,RTP_2050,8.0,1.0,23148,25104
48357,9,131.0,BRT_StateSt,192,23148,26488,0.09,602.00,RTP_2050,0.0,0.0,23148,26488
48358,9,131.0,BRT_StateSt,193,26488,26487,0.15,602.00,RTP_2050,23.0,1.0,26487,26488


,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,MODEL,COUNT,BAExist,NewA,NewB
5538,4,68.0,S455,355,3403,3440,0.06,134.02,BY_2019,0.0,0.0,3403,3440
6055,4,71.0,S470,283,3403,3440,0.06,78.21,BY_2019,0.0,0.0,3403,3440
13888,4,65.0,S455,355,3403,3440,0.06,151.88,TIP_2024,0.0,0.0,3403,3440
14409,4,68.0,S470,283,3403,3440,0.06,85.61,TIP_2024,0.0,0.0,3403,3440
22062,4,59.0,S455,355,3403,3440,0.06,154.45,RTP_2030,0.0,0.0,3403,3440
22582,4,62.0,S470,283,3403,3440,0.06,89.49,RTP_2030,0.0,0.0,3403,3440
31509,4,58.0,S455,355,3403,3440,0.06,93.78,RTP_2040,0.0,0.0,3403,3440
32029,4,61.0,S470,283,3403,3440,0.06,43.97,RTP_2040,0.0,0.0,3403,3440
41804,4,58.0,S455,355,3403,3440,0.06,92.77,RTP_2050,0.0,0.0,3403,3440
42324,4,61.0,S470,283,3403,3440,0.06,43.96,RTP_2050,0.0,0.0,3403,3440


,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,MODEL,COUNT,BAExist,NewA,NewB
5293,4,68.0,S455,110,3440,3403,0.06,78.27,BY_2019,10.0,1.0,3403,3440
5888,4,71.0,S470,116,3440,3403,0.06,175.69,BY_2019,10.0,1.0,3403,3440
13643,4,65.0,S455,110,3440,3403,0.06,84.98,TIP_2024,10.0,1.0,3403,3440
14242,4,68.0,S470,116,3440,3403,0.06,148.78,TIP_2024,10.0,1.0,3403,3440
21817,4,59.0,S455,110,3440,3403,0.06,75.91,RTP_2030,10.0,1.0,3403,3440
22415,4,62.0,S470,116,3440,3403,0.06,137.16,RTP_2030,10.0,1.0,3403,3440
31264,4,58.0,S455,110,3440,3403,0.06,67.22,RTP_2040,10.0,1.0,3403,3440
31862,4,61.0,S470,116,3440,3403,0.06,77.79,RTP_2040,10.0,1.0,3403,3440
41559,4,58.0,S455,110,3440,3403,0.06,70.01,RTP_2050,10.0,1.0,3403,3440
42157,4,61.0,S470,116,3440,3403,0.06,82.00,RTP_2050,10.0,1.0,3403,3440


,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,MODEL,COUNT,BAExist,NewA,NewB
7717,6,100.0,S456,131,3611,3503,0.17,30.08,BY_2019,0.0,0.0,3611,3503
16218,6,100.0,S456,131,3611,3503,0.17,45.29,TIP_2024,0.0,0.0,3611,3503
25343,6,98.0,S456,131,3611,3503,0.17,48.60,RTP_2030,0.0,0.0,3611,3503
35289,6,101.0,S456,131,3611,3503,0.17,59.66,RTP_2040,0.0,0.0,3611,3503
46637,6,115.0,S456,131,3611,3503,0.17,77.80,RTP_2050,0.0,0.0,3611,3503


In [20]:
#group by Mode, Model A, B

dfLinkTotalsByMode = dfTotVol_NewAB.groupby(['MODEL','MODE','NewA','NewB'],as_index=False).agg(TOT_VOL=('D_TOT_VOL','sum'),COUNT=('D_TOT_VOL','size'))
display(dfLinkTotalsByMode)

#should have values
display(dfLinkTotalsByMode[(dfLinkTotalsByMode['NewA']==3403) & (dfLinkTotalsByMode['NewB']==3440)])

#should be empty
display(dfLinkTotalsByMode[(dfLinkTotalsByMode['NewB']==3403) & (dfLinkTotalsByMode['NewA']==3440)])

#check total
display(dfLinkTotalsByMode['TOT_VOL'].sum())

,MODEL,MODE,NewA,NewB,TOT_VOL,COUNT
0,BY_2019,4,3403,3440,466.19,4
1,BY_2019,4,3403,3449,466.19,4
2,BY_2019,4,3406,10920,466.19,4
3,BY_2019,4,3406,10921,470.80,4
4,BY_2019,4,3407,8979,35.31,1
...,...,...,...,...,...,...
28557,TIP_2024,9,26401,22904,3198.45,1
28558,TIP_2024,9,26420,22830,3045.28,1
28559,TIP_2024,9,26487,26488,3837.59,2
28560,TIP_2024,9,31412,9550,954.12,1


,MODEL,MODE,NewA,NewB,TOT_VOL,COUNT
0,BY_2019,4,3403,3440,466.19,4
4772,RTP_2030,4,3403,3440,457.01,4
10478,RTP_2040,4,3403,3440,282.76,4
16578,RTP_2050,4,3403,3440,288.74,4
23604,TIP_2024,4,3403,3440,471.25,4


,MODEL,MODE,NewA,NewB,TOT_VOL,COUNT


19624960.73

In [21]:
dfLinkTotalsByModeTotal = dfLinkTotalsByMode.groupby(['MODEL','NewA','NewB'],as_index=False).agg(TOT_VOL=('TOT_VOL','sum'),MODECOUNT=('COUNT','size'))
dfLinkTotalsByModeTotal['MODE']=0 # 0=All Modes
display(dfLinkTotalsByModeTotal)
display(dfLinkTotalsByModeTotal['TOT_VOL'].sum())

#check link with 4 modes
display(dfLinkTotalsByMode[dfLinkTotalsByMode['NewA']==31412])

dfLinkTotals = dfLinkTotalsByModeTotal[['MODEL','MODE','NewA','NewB','TOT_VOL']].append(dfLinkTotalsByMode[['MODEL','MODE','NewA','NewB','TOT_VOL']])
display(dfLinkTotals)
display(dfLinkTotals['TOT_VOL'].sum()/2)


#check LRT in Murray
dfLinkTotals[(dfLinkTotals['NewA']==20013) & (dfLinkTotals['NewB']==20016)]

,MODEL,NewA,NewB,TOT_VOL,MODECOUNT,MODE
0,BY_2019,1785,11253,0.02,1,0
1,BY_2019,3403,3440,466.19,1,0
2,BY_2019,3403,3449,466.19,1,0
3,BY_2019,3406,10920,466.19,1,0
4,BY_2019,3406,10921,470.80,1,0
...,...,...,...,...,...,...
23749,TIP_2024,31141,21138,82.82,1,0
23750,TIP_2024,31145,23121,0.97,1,0
23751,TIP_2024,31188,21137,82.82,1,0
23752,TIP_2024,31412,9550,955.26,2,0


19624960.729999997

,MODEL,MODE,NewA,NewB,TOT_VOL,COUNT
3992,BY_2019,4,31412,9550,0.76,1
3993,BY_2019,4,31412,31413,1253.34,6
4527,BY_2019,6,31412,31413,43.65,2
8386,RTP_2030,4,31412,9550,0.88,1
8387,RTP_2030,4,31412,31413,354.47,5
9929,RTP_2030,6,31412,31413,76.32,2
10476,RTP_2030,9,31412,9550,960.74,1
10477,RTP_2030,9,31412,31413,960.74,1
14068,RTP_2040,4,31412,9550,1.34,1
14069,RTP_2040,4,31412,31413,434.12,5


,MODEL,MODE,NewA,NewB,TOT_VOL
0,BY_2019,0,1785,11253,0.02
1,BY_2019,0,3403,3440,466.19
2,BY_2019,0,3403,3449,466.19
3,BY_2019,0,3406,10920,466.19
4,BY_2019,0,3406,10921,470.80
...,...,...,...,...,...
28557,TIP_2024,9,26401,22904,3198.45
28558,TIP_2024,9,26420,22830,3045.28
28559,TIP_2024,9,26487,26488,3837.59
28560,TIP_2024,9,31412,9550,954.12


19624960.729999997

,MODEL,MODE,NewA,NewB,TOT_VOL
3278,BY_2019,0,20013,20016,12507.27
7504,RTP_2030,0,20013,20016,10378.16
12256,RTP_2040,0,20013,20016,11934.48
17525,RTP_2050,0,20013,20016,16014.48
22648,TIP_2024,0,20013,20016,11120.75
4546,BY_2019,7,20013,20016,12507.27
9948,RTP_2030,7,20013,20016,10378.16
15946,RTP_2040,7,20013,20016,11934.48
22937,RTP_2050,7,20013,20016,16014.48
28244,TIP_2024,7,20013,20016,11120.75


In [22]:
# pivot table for mapping

dfLinkTotals['COL'] = 'M' + dfLinkTotals['MODE'].astype(str) + '_' + dfLinkTotals['MODEL'].str[-2:]
dfLinkTotals['LINKID'] = dfLinkTotals['NewA'].astype(str) + '_' + dfLinkTotals['NewB'].astype(str)
dfLinkTotals['TOT_VOL'] = dfLinkTotals['TOT_VOL'].astype(int)

dfLinkTotals_pivot = dfLinkTotals.pivot_table(values='TOT_VOL',index=('LINKID'),columns=('COL'))
dfLinkTotals_pivot = dfLinkTotals_pivot.fillna(0)
dfLinkTotals_pivot = dfLinkTotals_pivot.apply(pd.to_numeric).astype(int) # convert all columns of DataFrame
dfLinkTotals_pivot = dfLinkTotals_pivot.reset_index()
dfLinkTotals_pivot

COL,LINKID,M0_19,M0_24,M0_30,M0_40,M0_50,M4_19,M4_24,M4_30,M4_40,...,M8_19,M8_24,M8_30,M8_40,M8_50,M9_19,M9_24,M9_30,M9_40,M9_50
0,10003_10339,16,18,30,39,45,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10003_8410,22,26,32,41,58,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10005_10338,16,18,30,39,45,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10011_9461,610,760,1102,1210,1279,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10013_11185,594,741,1071,1170,1233,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6082,9983_5139,101,100,116,123,120,101,100,116,123,...,0,0,0,0,0,0,0,0,0,0
6083,9985_5428,138,147,0,0,0,138,147,0,0,...,0,0,0,0,0,0,0,0,0,0
6084,9991_5136,0,0,16,28,603,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6085,9995_5132,0,0,0,0,812,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
#check Frontrunner Link just north of SL Central
dfLinkTotals_pivot[dfLinkTotals_pivot['LINKID']=='20209_20227']

COL,LINKID,M0_19,M0_24,M0_30,M0_40,M0_50,M4_19,M4_24,M4_30,M4_40,...,M8_19,M8_24,M8_30,M8_40,M8_50,M9_19,M9_24,M9_30,M9_40,M9_50
757,20209_20227,6983,8169,6853,8390,10519,0,0,0,0,...,6983,8169,6853,8390,10519,0,0,0,0,0


In [24]:
#check Light Rail Link just north of Murray Central
dfLinkTotals_pivot[dfLinkTotals_pivot['LINKID']=='20013_20016']

COL,LINKID,M0_19,M0_24,M0_30,M0_40,M0_50,M4_19,M4_24,M4_30,M4_40,...,M8_19,M8_24,M8_30,M8_40,M8_50,M9_19,M9_24,M9_30,M9_40,M9_50
581,20013_20016,12507,11120,10378,11934,16014,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# export to csv
dfLinkTotals_pivot.to_csv(os.path.join(results_folder,'linkTotals.csv'),index=False)

In [31]:
#create shapefile
sdfLinks = pd.DataFrame.spatial.from_featureclass(masternetwork)

sdfLinks = sdfLinks[sdfLinks['FT_2019']>1]
sdfLinks = sdfLinks[['A','B','LINKID','SHAPE','FT_2019','FT_2024','FT19_30','FT19_40','FT19_50']]

sdfLinksWithTransit = pd.DataFrame.merge(sdfLinks,dfLinkTotals_pivot,on='LINKID',how='left')
sdfLinksWithTransit = sdfLinksWithTransit.fillna(0)


sdfLinksWithTransit['M0_50m19'] = sdfLinksWithTransit['M0_50'] - sdfLinksWithTransit['M0_19']
sdfLinksWithTransit['M0_24m19'] = sdfLinksWithTransit['M0_24'] - sdfLinksWithTransit['M0_19']
sdfLinksWithTransit['M0_30m24'] = sdfLinksWithTransit['M0_30'] - sdfLinksWithTransit['M0_24']
sdfLinksWithTransit['M0_40m30'] = sdfLinksWithTransit['M0_40'] - sdfLinksWithTransit['M0_30']
sdfLinksWithTransit['M0_50m40'] = sdfLinksWithTransit['M0_50'] - sdfLinksWithTransit['M0_40']

sdfLinksWithTransit['M4_50m19'] = sdfLinksWithTransit['M4_50'] - sdfLinksWithTransit['M4_19']
sdfLinksWithTransit['M4_24m19'] = sdfLinksWithTransit['M4_24'] - sdfLinksWithTransit['M4_19']
sdfLinksWithTransit['M4_30m24'] = sdfLinksWithTransit['M4_30'] - sdfLinksWithTransit['M4_24']
sdfLinksWithTransit['M4_40m30'] = sdfLinksWithTransit['M4_40'] - sdfLinksWithTransit['M4_30']
sdfLinksWithTransit['M4_50m40'] = sdfLinksWithTransit['M4_50'] - sdfLinksWithTransit['M4_40']

sdfLinksWithTransit['M5_50m19'] = sdfLinksWithTransit['M5_50'] - sdfLinksWithTransit['M5_19']
sdfLinksWithTransit['M5_24m19'] = sdfLinksWithTransit['M5_24'] - sdfLinksWithTransit['M5_19']
sdfLinksWithTransit['M5_30m24'] = sdfLinksWithTransit['M5_30'] - sdfLinksWithTransit['M5_24']
sdfLinksWithTransit['M5_40m30'] = sdfLinksWithTransit['M5_40'] - sdfLinksWithTransit['M5_30']
sdfLinksWithTransit['M5_50m40'] = sdfLinksWithTransit['M5_50'] - sdfLinksWithTransit['M5_40']

sdfLinksWithTransit['M6_50m19'] = sdfLinksWithTransit['M6_50'] - sdfLinksWithTransit['M6_19']
sdfLinksWithTransit['M6_24m19'] = sdfLinksWithTransit['M6_24'] - sdfLinksWithTransit['M6_19']
sdfLinksWithTransit['M6_30m24'] = sdfLinksWithTransit['M6_30'] - sdfLinksWithTransit['M6_24']
sdfLinksWithTransit['M6_40m30'] = sdfLinksWithTransit['M6_40'] - sdfLinksWithTransit['M6_30']
sdfLinksWithTransit['M6_50m40'] = sdfLinksWithTransit['M6_50'] - sdfLinksWithTransit['M6_40']

sdfLinksWithTransit['M7_50m19'] = sdfLinksWithTransit['M7_50'] - sdfLinksWithTransit['M7_19']
sdfLinksWithTransit['M7_24m19'] = sdfLinksWithTransit['M7_24'] - sdfLinksWithTransit['M7_19']
sdfLinksWithTransit['M7_30m24'] = sdfLinksWithTransit['M7_30'] - sdfLinksWithTransit['M7_24']
sdfLinksWithTransit['M7_40m30'] = sdfLinksWithTransit['M7_40'] - sdfLinksWithTransit['M7_30']
sdfLinksWithTransit['M7_50m40'] = sdfLinksWithTransit['M7_50'] - sdfLinksWithTransit['M7_40']

sdfLinksWithTransit['M8_50m19'] = sdfLinksWithTransit['M8_50'] - sdfLinksWithTransit['M8_19']
sdfLinksWithTransit['M8_24m19'] = sdfLinksWithTransit['M8_24'] - sdfLinksWithTransit['M8_19']
sdfLinksWithTransit['M8_30m24'] = sdfLinksWithTransit['M8_30'] - sdfLinksWithTransit['M8_24']
sdfLinksWithTransit['M8_40m30'] = sdfLinksWithTransit['M8_40'] - sdfLinksWithTransit['M8_30']
sdfLinksWithTransit['M8_50m40'] = sdfLinksWithTransit['M8_50'] - sdfLinksWithTransit['M8_40']

sdfLinksWithTransit['M9_50m19'] = sdfLinksWithTransit['M9_50'] - sdfLinksWithTransit['M9_19']
sdfLinksWithTransit['M9_24m19'] = sdfLinksWithTransit['M9_24'] - sdfLinksWithTransit['M9_19']
sdfLinksWithTransit['M9_30m24'] = sdfLinksWithTransit['M9_30'] - sdfLinksWithTransit['M9_24']
sdfLinksWithTransit['M9_40m30'] = sdfLinksWithTransit['M9_40'] - sdfLinksWithTransit['M9_30']
sdfLinksWithTransit['M9_50m40'] = sdfLinksWithTransit['M9_50'] - sdfLinksWithTransit['M9_40']


sdfLinksWithTransit['M0_50m19'] = sdfLinksWithTransit['M0_50m19'].astype(int)
sdfLinksWithTransit['M0_24m19'] = sdfLinksWithTransit['M0_24m19'].astype(int)
sdfLinksWithTransit['M0_30m24'] = sdfLinksWithTransit['M0_30m24'].astype(int)
sdfLinksWithTransit['M0_40m30'] = sdfLinksWithTransit['M0_40m30'].astype(int)
sdfLinksWithTransit['M0_50m40'] = sdfLinksWithTransit['M0_50m40'].astype(int)

sdfLinksWithTransit['M4_50m19'] = sdfLinksWithTransit['M4_50m19'].astype(int)
sdfLinksWithTransit['M4_24m19'] = sdfLinksWithTransit['M4_24m19'].astype(int)
sdfLinksWithTransit['M4_30m24'] = sdfLinksWithTransit['M4_30m24'].astype(int)
sdfLinksWithTransit['M4_40m30'] = sdfLinksWithTransit['M4_40m30'].astype(int)
sdfLinksWithTransit['M4_50m40'] = sdfLinksWithTransit['M4_50m40'].astype(int)

sdfLinksWithTransit['M5_50m19'] = sdfLinksWithTransit['M5_50m19'].astype(int)
sdfLinksWithTransit['M5_24m19'] = sdfLinksWithTransit['M5_24m19'].astype(int)
sdfLinksWithTransit['M5_30m24'] = sdfLinksWithTransit['M5_30m24'].astype(int)
sdfLinksWithTransit['M5_40m30'] = sdfLinksWithTransit['M5_40m30'].astype(int)
sdfLinksWithTransit['M5_50m40'] = sdfLinksWithTransit['M5_50m40'].astype(int)

sdfLinksWithTransit['M6_50m19'] = sdfLinksWithTransit['M6_50m19'].astype(int)
sdfLinksWithTransit['M6_24m19'] = sdfLinksWithTransit['M6_24m19'].astype(int)
sdfLinksWithTransit['M6_30m24'] = sdfLinksWithTransit['M6_30m24'].astype(int)
sdfLinksWithTransit['M6_40m30'] = sdfLinksWithTransit['M6_40m30'].astype(int)
sdfLinksWithTransit['M6_50m40'] = sdfLinksWithTransit['M6_50m40'].astype(int)

sdfLinksWithTransit['M7_50m19'] = sdfLinksWithTransit['M7_50m19'].astype(int)
sdfLinksWithTransit['M7_24m19'] = sdfLinksWithTransit['M7_24m19'].astype(int)
sdfLinksWithTransit['M7_30m24'] = sdfLinksWithTransit['M7_30m24'].astype(int)
sdfLinksWithTransit['M7_40m30'] = sdfLinksWithTransit['M7_40m30'].astype(int)
sdfLinksWithTransit['M7_50m40'] = sdfLinksWithTransit['M7_50m40'].astype(int)

sdfLinksWithTransit['M8_50m19'] = sdfLinksWithTransit['M8_50m19'].astype(int)
sdfLinksWithTransit['M8_24m19'] = sdfLinksWithTransit['M8_24m19'].astype(int)
sdfLinksWithTransit['M8_30m24'] = sdfLinksWithTransit['M8_30m24'].astype(int)
sdfLinksWithTransit['M8_40m30'] = sdfLinksWithTransit['M8_40m30'].astype(int)
sdfLinksWithTransit['M8_50m40'] = sdfLinksWithTransit['M8_50m40'].astype(int)

sdfLinksWithTransit['M9_50m19'] = sdfLinksWithTransit['M9_50m19'].astype(int)
sdfLinksWithTransit['M9_24m19'] = sdfLinksWithTransit['M9_24m19'].astype(int)
sdfLinksWithTransit['M9_30m24'] = sdfLinksWithTransit['M9_30m24'].astype(int)
sdfLinksWithTransit['M9_40m30'] = sdfLinksWithTransit['M9_40m30'].astype(int)
sdfLinksWithTransit['M9_50m40'] = sdfLinksWithTransit['M9_50m40'].astype(int)



display(sdfLinksWithTransit)

sdfLinksWithTransit.spatial.to_featureclass(os.path.join(results_folder,r'LinksWithRiders_v3.shp'),sanitize_columns=False)

,A,B,LINKID,SHAPE,FT_2019,FT_2024,FT19_30,FT19_40,FT19_50,M0_19,...,M8_50m19,M8_24m19,M8_30m24,M8_40m30,M8_50m40,M9_50m19,M9_24m19,M9_30m24,M9_40m30,M9_50m40
0,53,11455,53_11455,"{'paths': [[[416925.0, 4596274.0], [416664.460...",5,5,5,5,5,0.0,...,0,0,0,0,0,0,0,0,0,0
1,3401,3657,3401_3657,"{'paths': [[[410007.0, 4488724.0], [410016.0, ...",4,4,2,2,2,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3401,4366,3401_4366,"{'paths': [[[410007.0, 4488724.0], [410007.0, ...",4,4,2,2,2,0.0,...,0,0,0,0,0,0,0,0,0,0
3,3401,4567,3401_4567,"{'paths': [[[410007.0, 4488724.0], [410375.0, ...",5,5,5,5,5,0.0,...,0,0,0,0,0,0,0,0,0,0
4,3401,8084,3401_8084,"{'paths': [[[410007.0, 4488724.0], [409711.859...",5,5,5,5,5,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36117,31423,31425,21749_24604,"{'paths': [[[435364.95427833294, 4465605.09449...",42,13,13,13,42,0.0,...,0,0,0,0,0,0,0,0,0,0
36118,31424,31422,21751_21748,"{'paths': [[[435234.6050110983, 4465733.814391...",41,13,13,13,41,0.0,...,0,0,0,0,0,0,0,0,0,0
36119,31425,31416,21749_24604,"{'paths': [[[435276.96852294955, 4465769.66044...",42,13,13,13,42,0.0,...,0,0,0,0,0,0,0,0,0,0
36120,31426,22488,23667_22488,"{'paths': [[[433392.2709439084, 4467292.690812...",4,3,3,3,3,0.0,...,0,0,0,0,0,0,0,0,0,0


'e:\\GitHub\\Transit-Map\\results\\LinksWithRiders_v3.shp'